In [32]:
# Importaré todas las funciones de prácticas previas y librerias necesarias para la realización de los ejercicios
import numpy as np
import math

def LU_desc(A):
    """
    Esta función calcula la descomposición LU de Doolittle de una matriz A sin intercambio de filas.
    A debe ser un array de Numpy.
    """
    n=np.shape(A)[1]
    U=np.zeros([n,n])
    L=np.eye(n)
    
    for k in range(n-1):
        U[k,k]=A[k,k]-np.dot(L[k,:k],U[:k,k])
        for j in range(k+1,n):
            U[k,j]=A[k,j]-np.dot(L[k,:k],U[:k,j])
        for j in range(k+1):
            L[k+1,j]=(A[k+1,j]-np.dot(L[k+1,:j],U[:j,j]))/U[j,j]
    U[n-1,n-1]=A[n-1,n-1]-np.dot(L[n-1,:n-1],U[:n-1,n-1])
    return L, U

def sust_progresiva(A,b):
    """
    Esta función resuelve el sistema Ax=b mediante sustitución progresiva. 
    A y b deben ser arrays de numpy. 
    A debe ser una matriz triangular inferior.
    """
    A=A.astype(float)     # cambiamos los elementos a tipo flotante
    b=b.astype(float) 
    n=np.size(b)
    x=np.zeros(n)         # almacenaremos aquí la solución del sistema                  
    # ahora se resuelve el sistema por sustitución regresiva
    x[0]=b[0]/A[0,0]
    for i in range(1,n):
        restar = 0
        for j in range(0,i):
            restar += (x[j]*A[i,j])
        x[i]=(b[i]- restar)/A[i,i]
                
    return x

def sust_regresiva(A,b):
    """
    Esta función resuelve el sistema Ax=b mediante sustitución regresiva. 
    A y b deben ser arrays de numpy. 
    A debe ser una matriz triangular superior.
    """
    A=A.astype(float)     # cambiamos los elementos a tipo flotante
    b=b.astype(float) 
    n=np.size(b)
    x=np.zeros(n)         # almacenaremos aquí la solución del sistema
                       
    # ahora se resuelve el sistema por sustitución regresiva
    x[n-1]=b[n-1]/A[n-1,n-1]
    for i in range(n-2,-1,-1):     
        x[i]=(b[i]-np.dot(A[i][i+1:],x[i+1:]))/A[i,i]
                
    return x

def CHOL_desc(A):
    """
    Esta función calcula la descomposición Cholesky de una matriz A sin intercambio de filas.
    A debe ser un array de Numpy, una matriz simétrica y definida positiva.
    """
    n=np.shape(A)[1]
    L=np.zeros([n,n])
    
    for i in range(n):
        for j in range(i+1):
            suma=0
            if (j==i):
                for k in range(j+1):
                    suma += pow(L[j,k], 2)
                if ( suma <= A[j,j]):
                    L[j,j] = math.sqrt(A[j,j]-suma)
                else: print("ERROR: matriz no definida positiva, descomposición fallida"); return np.zeros([n,n])
            else:
                for k in range(j+1):
                    suma += L[i,k]*L[j,k]
                L[i,j] = (A[i,j]-suma)/L[j,j]
                
    return L

def Gauss(A,b):
    """
    Esta función resuelve el sistema Ax=b mediante el método de Gauss. 
    A y b deben ser arrays de numpy.
    """
    A=A.astype(float)     # cambiamos los elementos a tipo flotante
    bb=(np.array([b]).transpose()).astype(float) 
    Ab=np.concatenate((A,bb),axis=1)
    n=np.size(b)
    x=np.zeros(n)         # almacenaremos aquí la solución del sistema
    
    # aquí se hacen operaciones por filas en la matriz para conseguir una triangular
    for i in range(n):
        if Ab[i,i]==0: sys.exit('Aparece una división por cero, el proceso no puede concluirse')
        else:
            for j in range(i+1,n):
                m=Ab[j,i]/Ab[i,i]
                Ab[j][i+1:]=Ab[j][i+1:]-m*Ab[i][i+1:]
                    
    # ahora se resuelve el sistema por sustitución regresiva
    x[n-1]=Ab[n-1,n]/Ab[n-1,n-1]
    for i in range(n-2,-1,-1):
        x[i]=(Ab[i,n]-np.dot(Ab[i][i+1:n],x[i+1:n]))/Ab[i,i]
                
    return x

def Gauss_Piv(A,b):
    """
    Esta función resuelve el sistema Ax=b mediante el método de Gauss. 
    A y b deben ser arrays de numpy.
    """
    A=A.astype(float)     # cambiamos los elementos a tipo flotante
    bb=(np.array([b]).transpose()).astype(float) 
    Ab=np.concatenate((A,bb),axis=1)
    n=np.size(b)
       
    # aquí se hacen operaciones por filas en la matriz para conseguir una triangular
    # se seguirá el métododo de pivote parcial, por el cual buscaré el más adecuado de entre el resto de elementos de la columna
    for i in range(n):
        maximo = i
        for k in range(i,n):
            if Ab[k,i] > Ab[maximo,i]:
                maximo = k
        Ab[[i,maximo], :] = Ab[[maximo,i], :]
        if Ab[i,i]==0: sys.exit('Aparece una división por cero, el proceso no puede concluirse')
        else:
            for j in range(i+1,n):
                m=Ab[j,i]/Ab[i,i]
                Ab[j]=Ab[j]-m*Ab[i]
                    
    # ahora se resuelve el sistema por sustitución regresiva
    AG=Ab[:,:n]
    bG=Ab[:,n].transpose()
    x=sust_regresiva(AG,bG)
                
    return x

def Jacobi(A, b, maxiter=1000, tol=10**(-10)):
    """
    Esta función resuelve el sistema Ax=b de forma aproximada mediante el método iterativo de Jacobi empezando las iteraciones
    en el vector nulo.
    Las matrices A y b deben ser arrays de NumPy.
    El número máximo de iteraciones se puede introducir por el usuario y es 1000 por defecto.
    La tolerancia también se puede introducir por el usuario y es 10^(-10) por defecto.
    El método se interrumpe cuando se alcanza el número máximo de iteraciones o cuando la tolerancia es menor que la indicada.
    
    """
    
    n=np.shape(A)[1] # se calcula la dimensión de A
    d=np.diag(A)     # obtenemos un array con los elementos de la diagonal de la matriz A
    D=np.diag(d)     # creamos un array diagonal con los elementos de d
    R=A-D            # matriz con los elementos que no están en la diagonal
    x=np.zeros(n)
    y=np.zeros(n)
          
    if np.prod(d)==0:
        print('Hay elementos nulos en la diagonal de la matriz')
    else:
        k=1
        er=1
        nx=1
        while (er>tol*nx and k<maxiter):
            nx=np.linalg.norm(x,1)        
            y=(b - np.dot(R,x)) / d       # aplicamos una iteración del método de Jacobi obteniendo un vector y
            er=np.linalg.norm(y-x,1)      # calculamos la norma de la diferencia entre y y el resultado de la iteración anterior        
            x=y
            # print('Iteración k=', k, '  x^(k)= ', x ) # descomentar si se quieren ver las iteraciones.
            k+=1
        return x

**Ejercicio 1:** Modifica el código anterior para resolver de forma aproximada un sistema de ecuaciones lineales utilizando el método de **Gauss-Seidel**. Comprueba el funcionamiento del programa con el sistema $A x=b$ donde
$$A=\left(\begin{array}{rrr}
10 & 4 & 1 \\
4 & 10 & 1 \\
1 & 1 & 5 \end{array} \right) \qquad b=\left(15,15,7 \right) $$ 


In [5]:
def Gauss_Seidel(A, b, maxiter=1000, tol=10**(-10)):
    """
    Esta función resuelve el sistema Ax=b de forma aproximada mediante el método iterativo de Gauss-Seidel empezando las iteraciones
    en el vector nulo.
    Las matrices A y b deben ser arrays de NumPy.
    El número máximo de iteraciones se puede introducir por el usuario y es 1000 por defecto.
    La tolerancia también se puede introducir por el usuario y es 10^(-10) por defecto.
    El método se interrumpe cuando se alcanza el número máximo de iteraciones o cuando la tolerancia es menor que la indicada.
    
    """
    
    n=np.shape(A)[1] # se calcula la dimensión de A
    Q=np.tril(A,0)   # obtenemos un array con los elementos de la diagonal y de la triangular inferior de la matriz A,
                     # que será nuestra matriz de iteracción
    d=np.diag(A)
    U=A-Q            # matriz con los elementos de la triangular superior
    x=np.zeros(n)
    y=np.zeros(n)
          
    if np.prod(d)==0:
        print('Hay elementos nulos en la diagonal de la matriz')
    else:
        k=1
        er=1
        nx=1
        while (er>tol*nx and k<maxiter):
            nx=np.linalg.norm(x,1)        
            y=np.dot(np.linalg.inv(Q),(b - np.dot(U,x)))        # aplicamos una iteración del método de Gauss-Seidel obteniendo un vector y
            er=np.linalg.norm(y-x,1)      # calculamos la norma de la diferencia entre y y el resultado de la iteración anterior        
            x=y
            # print('Iteración k=', k, '  x^(k)= ', x ) # descomentar si se quieren ver las iteraciones.
            k+=1
        return x
    

In [6]:
A=np.array([[10,4,1],[4,10,1],[1,1,5]])
b=np.array([15,15,7])
Gauss_Seidel(A,b)

array([1., 1., 1.])

**Ejercicio 2 (opcional):** Una vez programado Gauss-Seidel, haz las modificaciones oportunas para definir una función que resuelva un sistema utilizando un método de relajación con la constante $\omega$ introducida por el usuario.

In [7]:
def Relajacion(A, b, w=1, maxiter=1000, tol=10**(-10)):
    """
    Esta función resuelve el sistema Ax=b de forma aproximada mediante el método iterativo de Relajación empezando las iteraciones
    en el vector nulo.
    Las matrices A y b deben ser arrays de NumPy.
    El número máximo de iteraciones se puede introducir por el usuario y es 1000 por defecto.
    La tolerancia también se puede introducir por el usuario y es 10^(-10) por defecto.
    Se deberá introducir el parametro w que intervendrá en las iteracciones y es 1 por defecto.
    El método se interrumpe cuando se alcanza el número máximo de iteraciones o cuando la tolerancia es menor que la indicada.
    
    """
    
    n=np.shape(A)[1] # se calcula la dimensión de A
    L=np.tril(A,-1)   # obtenemos un array con los elementos de la diagonal y de la triangular inferior de la matriz A,
                     # que será nuestra matriz de iteracción
    d=np.diag(A)
    D=np.diag(d)
    U=A-L-D            # matriz con los elementos de la triangular superior
    Q=D+(w*L)
    
    x=np.zeros(n)
    y=np.zeros(n)
          
    if np.prod(d)==0:
        print('Hay elementos nulos en la diagonal de la matriz')
    else:
        k=1
        er=1
        nx=1
        while (er>tol*nx and k<maxiter):
            nx=np.linalg.norm(x,1)
             # aplicamos una iteración del método de relajación obteniendo un vector y
            y=np.dot(w*np.linalg.inv(Q),(b - np.dot(U,x)))+((1-w)*np.dot(np.dot(np.linalg.inv(Q),D),x))       
            er=np.linalg.norm(y-x,1)      # calculamos la norma de la diferencia entre y y el resultado de la iteración anterior        
            x=y
            # print('Iteración k=', k, '  x^(k)= ', x ) # descomentar si se quieren ver las iteraciones.
            k+=1
        return x

# Para terminar

Ahora ya tienes una función definida para cada uno de los métodos de resolución que hemos visto en clase. Estaría bien que los probaramos todos con un ejemplo un poco más grande ¿no crees? 

**Ejercicio final:** Define una matriz A cuadrada de orden 20 cuyo elemento $a_{ij}=1/(i+j)$ siempre que $i\ne j$ y $a_{ii}=20 *i^2$. Define también un vector b cuyas componentes sean todas iguales a 20. Resuelve ahora el sistema $A x=b$ con **todos los métodos** que hemos visto, es decir:

- Utilizando la función `solve` del módulo `linalg` de `NumPy`. 
- Mediante los método de **Gauss** y **Gauss con pivotaje parcial**.
- Utilizando descomposición **LU** y de **Cholesky**.
- De forma aproximada mediante los métodos de **Jacobi** y **Gauss-Seidel** con un máximo de 10000 iteraciones y una tolerancia de 10^(-12). Si también has programado el método de relajación, inclúyelo también utilizando $w=0.8$.

Comenta los resultados.

In [39]:
# Declaración de la matriz A y b
A=np.zeros((20,20))
for i in range(20):
    for j in range(20):
        if i==j:
            A[i,j]=20*((i+1)**2)
        else:
            A[i,j]=1/(i+j+2)
b=np.zeros(20)
for i in range(20): b[i]=20

# Resolución por distintos métodos
    
print("RESOLUCIÓN DEL SISTEMA:\n")

print("\tFuncion solve del modulo linalg de Numpy:\n")
x=np.linalg.solve(A,b)
print(x,"\n")

print("\tMétodo de Gauss:\n")
x=Gauss(A,b)
print(x,"\n")

print("\tMétodo de Gauss con pivote parcial:\n")
x=Gauss_Piv(A,b)
print(x,"\n")

print("\tUtilizando la descomposición LU:\n")
L,U = LU_desc(A)
y=sust_progresiva(L,b)
x=sust_regresiva(U,y)
# print("Matriz L:\n", L, "\n")      # Descomentar si quiere que se muestre la descomposición realizada
# print("Matriz U:\n", U, "\n")
# print("Solucion a Ly=b:\n",y, "\n")
print(x,"\n")

print("\tUtilizando la descomposición Cholesky:\n")
L=CHOL_desc(A)
y=sust_progresiva(L,b)
x=sust_regresiva(np.transpose(L),y)
# print("Matriz L:\n", L, "\n")      # Descomentar si quiere que se muestre la descomposición realizada
# print("Solucion a Ly=b:\n",y, "\n")
print(x,"\n")

print("\tMétodo recursivo de Jacobi:\n")
x=Jacobi(A,b,10000,10**(-12))
print(x,"\n")

print("\tMétodo recursivo de Gauss-Seidel:\n")
x=Gauss_Seidel(A,b,10000,10**(-12))
print(x,"\n")

print("\tMétodo recursivo de Relajación:\n")
x=Relajacion(A,b,0.8,10000,10**(-12))
print(x,"\n\n")

print("Como vemos, las soluciones al sistema planteado obtenidas atraves de distintos métodos coinciden.\n")
print("Por ello, es seguro afirmar que los resultados obtenidos son correctos.")

RESOLUCIÓN DEL SISTEMA:

	Funcion solve del modulo linalg de Numpy:

[0.99293021 0.24523958 0.10932211 0.06165761 0.03953924 0.02749887
 0.02022665 0.01550028 0.0122563  0.00993376 0.00821401 0.00690512
 0.00588592 0.00507681 0.00442377 0.0038891  0.00344582 0.00307423
 0.00275967 0.00249103] 

	Método de Gauss:

[0.99293021 0.24523958 0.10932211 0.06165761 0.03953924 0.02749887
 0.02022665 0.01550028 0.0122563  0.00993376 0.00821401 0.00690512
 0.00588592 0.00507681 0.00442377 0.0038891  0.00344582 0.00307423
 0.00275967 0.00249103] 

	Método de Gauss con pivote parcial:

[0.99293021 0.24523958 0.10932211 0.06165761 0.03953924 0.02749887
 0.02022665 0.01550028 0.0122563  0.00993376 0.00821401 0.00690512
 0.00588592 0.00507681 0.00442377 0.0038891  0.00344582 0.00307423
 0.00275967 0.00249103] 

	Utilizando la descomposición LU:

[0.99293021 0.24523958 0.10932211 0.06165761 0.03953924 0.02749887
 0.02022665 0.01550028 0.0122563  0.00993376 0.00821401 0.00690512
 0.00588592 0.00507681 0